# Model Errors (by strike and maturity)

In this notebook we compare different versions of the Heston MC model, and calculate model errors vs closed form solution.
We look at model errors at different strikes and maturities.

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
from qablet_contracts.eq.vanilla import Option
from qablet_contracts.timetable import py_to_ts

from src.models.aes import HestonAESMC
from src.models.basic import HestonMCBasic
from src.models.better import HestonMCBetter
from src.models.closed import price_vanilla_call
from src.qablet_utils import option_prices

## Create Dataset
Create the dataset, with MC params, discounts and fwds as required by the [Dataset API](https://qablet-academy.github.io/intro/dataset/)

In [ ]:
# Rate Curve
times = np.array([0.0, 5.0])
rates = np.array([0.1, 0.1])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

# Forward Curve
ticker = "EQ"
spot = 100.0
div_rate = 0.0
fwds = spot * np.exp((rates - div_rate) * times)
fwd_data = ("FORWARDS", np.column_stack((times, fwds)))

pricing_datetime = datetime(2023, 12, 31)
dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(pricing_datetime).value,
    "ASSETS": {"USD": discount_data, ticker: fwd_data},
    "MC": {
        "PATHS": 10_000,
        "TIMESTEP": 1 / 1000,
        "SEED": 1,
    },
    "HESTON": {
        "ASSET": ticker,
        "INITIAL_VAR": 0.04,
        "LONG_VAR": 0.04,
        "VOL_OF_VAR": 0.8,
        "MEANREV": 1.0,
        "CORRELATION": -0.9,
    },
}

## Calculate Single Option Prices

Define a [Vanilla Option](https://qablet.github.io/qablet-contracts/examples/equity_vanilla/) and calculate price.

In [ ]:
# Create Contract
strike = 100
timetable = Option(
    "USD", ticker, strike=strike, maturity=datetime(2024, 12, 31), is_call=True
).timetable()
print(timetable["events"].to_pandas())

  track                      time op  quantity unit
0       2024-12-31 00:00:00+00:00  >       0.0  USD
1       2024-12-31 00:00:00+00:00  +    -100.0  USD
2       2024-12-31 00:00:00+00:00  +       1.0   EQ


In [ ]:
# Create models and price
models = {
    "Basic": HestonMCBasic(),
    "AES": HestonAESMC(),
    "Better": HestonMCBetter(),  # Uses Log Euler for spot, antithetic paths, and milstein discretization for vol
}

for model_name, model in models.items():
    price, _ = model.price(timetable, dataset)
    print(f"{model_name:20s}: {price:11.6f}")

Basic               :   13.005833
AES                 :   12.879431
Better              :   13.005703


In [ ]:
# Closed form price
ref_price, _ = price_vanilla_call(
    strike,
    1.0,
    ticker,
    dataset,
)
print(f"ref_price: {ref_price:11.6f}")

ref_price:   12.903195


## Model Errors vs Closed Form

In [ ]:
expirations = [
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 12, 31),
]
strikes = np.array([0.8, 0.9, 1.0, 1.1, 1.2]) * spot
is_call = True

### Get Closed Form prices

In [ ]:
ref_price_df = pd.DataFrame.from_dict({"Strike": strikes})
for i, exp in enumerate(expirations):
    prc_ts = dataset["PRICING_TS"]
    # Get Time in years from the millisecond timestamps
    T = (py_to_ts(exp).value - prc_ts) / (365.25 * 24 * 3600 * 1e3)
    ref_prices = []
    for strike in strikes:
        ref_price, _ = price_vanilla_call(
            strike,
            T,
            ticker,
            dataset,
        )
        ref_prices.append(ref_price)

    exp_str = exp.strftime("%Y-%m-%d")
    ref_price_df[exp_str] = ref_prices

print(ref_price_df)

   Strike  2024-03-31  2024-06-30  2024-12-31
0    80.0   22.288457   24.694725   28.978834
1    90.0   13.191565   15.984275   20.740817
2   100.0    5.104238    7.921329   12.922584
3   110.0    0.215422    1.502994    5.898664
4   120.0    0.001162    0.041960    1.026916


In [ ]:
for model_name, model in models.items():
    prices_df = option_prices(
        ticker, expirations, strikes, is_call, model, dataset
    )
    print(f"\n{model_name}")
    print(prices_df - ref_price_df)


Basic
   Strike  2024-03-31  2024-06-30  2024-12-31
0     0.0    0.177418    0.209874    0.177377
1     0.0    0.155712    0.154603    0.147418
2     0.0    0.056505    0.074979    0.103383
3     0.0   -0.000935    0.010545    0.069230
4     0.0   -0.000815   -0.002394    0.001727

AES
   Strike  2024-03-31  2024-06-30  2024-12-31
0     0.0   -0.058169    0.007420    0.116343
1     0.0   -0.053802   -0.007827    0.075223
2     0.0   -0.035044   -0.017253    0.035863
3     0.0   -0.000447    0.019979    0.024081
4     0.0   -0.000500    0.004394    0.023946

Better
   Strike  2024-03-31  2024-06-30  2024-12-31
0     0.0    0.111926    0.097992    0.109349
1     0.0    0.060244    0.072703    0.096665
2     0.0    0.054248    0.051776    0.080560
3     0.0    0.010100    0.026519    0.044196
4     0.0    0.000927    0.000267    0.017338
